# Module 6 Challenge

# 3. Create a new set of 2,000 random latitudes and longitudes.

In [ ]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Create a set of random latitude and longitude combinations.
# lat run from 0 to 90 and lng from 0 to 180
# size of output is 2000
# zip lat and lngs together
# zip object will be in memory creating a tuple with index

lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)
lat_lngs

In [ ]:
# unpack lat_lngs zip object into a list
#  NOTE:  CAN ONLY UNZIP TUPLE ONCE BEFORE IT IS REMOVED FROM COMOPUTER MEMORY
#       MAKE SURE UNZIP INTO LIST BEFORE MOVING ON


# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [ ]:
len(coordinates)

# 4. Get the nearest city using the citipy module.

In [ ]:
# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy

In [ ]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    # using not in to determine whether the found city is already in the
    # cities list.  If not then append to add it (to avoid duplicates)
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

# 5. Perform an API call with the OpenWeatherMap.

In [ ]:
# Import the requests library.
import requests

# Import the API key.
from config import weather_api_key

In [ ]:
# Starting URL for Weather Map API Call.
# units= Imperial    standard, metric, imperial options
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
# print(url)

In [ ]:
####### only added to find json structure

######## Test the json ouput for the city of Axim to get json
# Create an endpoint URL for a city.
#city_url = url + "&q=" + "Axim"
#city_weather = requests.get(city_url)
#print(city_url)  ####### can print this url, click on it and review in Chrome to also see the json format

In [ ]:
###### only added to find json structure

####### print the json so can see the structure - still need to find WEATHER DESCRIPTION
#city_weather.json()

# 6. Retrieve the following information from the API call:
        Latitude and Longitude
        Maximum temperature
        Percent Humidity
        Percent cloudiness
        Wind Speed
        Weather Description (clouds, fog, light rain, clear sky)

In [ ]:
# Import the datetime module from the datetime library.
from datetime import datetime

In [ ]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters. Initialized at 1 becasue want the first iteration of 
# logging for each recorded response and set to start at 1
record_count = 1
set_count = 1

# Loop through all the cities in the list. 
# using enumerate() get index of the city for logging purposes
# city for creating an endpoint URL
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    # if remainder of index divided by 50 is equal to 0
    # and if index greater than or equal to 50
    # if true then set count and record count increment by 1
    
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    # removing blank spaces in the city name and concat city name
    
    city_url = url + "&q=" + city.replace(" ","+")
  
    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count. before the next city is processed
    record_count += 1
    
    # ADD TRY-EXCEPT TO THE RUN
    
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_wx_desc = city_weather["weather"][0]["description"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date,
                          "Current Description": city_wx_desc})

# If an error is experienced, skip the city.
# pass statement general purpose to handle all errors encountered and allow
# program to continue
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")
    
    

In [ ]:
# how many rows of data did we just spit out

len(city_data)

# 7. Add the data to a new DataFrame.

In [ ]:
# Convert the array of dictionaries to a Pandas DataFrame.
# city_data was just created above
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

In [ ]:
# put columns in the right order

new_column_order = ["City", "Country", "Lat", "Lng", "Max Temp",
                    "Humidity", "Cloudiness", "Wind Speed", "Current Description"]
city_data_df = city_data_df[new_column_order]
city_data_df.head(10)

# 8. Export the DataFrame as a CSV file, and save it as WeatherPy_Database.csv in the Weather_Database folder.

In [ ]:
# Create the output file (CSV).
output_data_file = "weather_database/WeatherPy_Database.csv"

# Export the WeatherPy_Database data into a CSV.
# with index label (or column A) header of City_ID
city_data_df.to_csv(output_data_file, index_label="City_ID")